In [ ]:
from os.path import expanduser
configuration_path = expanduser('~/Projects/crosscompute-examples/serve.yml')
configuration_path

In [ ]:
from crosscompute.routines.configuration import load_configuration
configuration = load_configuration(configuration_path)
configuration

In [ ]:
from os.path import dirname
configuration_folder = dirname(configuration_path)
configuration_folder

In [ ]:
configuration['imports']

In [ ]:
from crosscompute.constants import BATCH_ROUTE
from os.path import basename

def get_batch_definitions(configuration):
    batch_definitions = []
    for batch_definition in configuration.get('batches', []):
        try:
            batch_folder = batch_definition['folder']
        except KeyError:
            logging.error('folder required for each batch')
            continue
        batch_name = batch_definition.get(
            'name', basename(batch_folder))
        batch_slug = batch_definition.get(
            'slug', get_slug_from_name(batch_name))
        batch_uri = BATCH_ROUTE.format(
            batch_slug=batch_slug)
        batch_definitions.append({
            'name': batch_name,
            'slug': batch_slug,
            'uri': batch_uri,
            'folder': batch_folder,
        })
    return batch_definitions

In [ ]:
from crosscompute.constants import AUTOMATION_NAME, AUTOMATION_ROUTE
from crosscompute.macros import get_slug_from_name
from os.path import join

def get_automation_definitions(configuration, configuration_folder):
    automation_configurations = [configuration]
    for import_configuration in configuration.get('imports', []):
        if 'path' in import_configuration:
            c = load_configuration(join(configuration_folder, import_configuration['path']))
        else:
            continue
        automation_configurations.append(c)
        
    automation_definitions = []
    for automation_configuration in automation_configurations:
        if 'output' not in automation_configuration:
            continue
        automation_name = automation_configuration.get(
            'name', AUTOMATION_NAME.format(automation_index=0))
        automation_slug = automation_configuration.get(
            'slug', get_slug_from_name(automation_name))
        automation_uri = AUTOMATION_ROUTE.format(
            automation_slug=automation_slug)
        automation_definitions.append({
            'name': automation_name,
            'slug': automation_slug,
            'uri': automation_uri,
            'batches': get_batch_definitions(automation_configuration),
        })
    return automation_definitions

automation_definitions = get_automation_definitions(configuration, configuration_folder)
automation_definitions